In [4]:
from cegalprizm.pycoderunner import WorkflowDescription

In [5]:
# Start: PWR Description



pwr_description = WorkflowDescription(name="Fault prediction",
                                      category="Seismic interpretation ML",
                                      description="Use this workflow to create a  fault model",
                                      authors="sebastian.grebe@cegal.com",
                                      version="0.1")


pwr_description.add_string_parameter("string_value", "String Value2", "A string input", "hello")
pwr_description.add_object_ref_parameter('seismic_id','seismic','Select a seismic cube','seismic_3d')
#pwr_description.add_object_ref_parameter("borehole_id2", "Borehol2e", "A borehole", "boreholes")
#pwr_description.add_object_ref_parameter("seis_horizon", "Horizon", "A horizon", "seismic_horizon")


# End: PWR Description

In [ ]:
if 'parameters' not in locals() and 'parameters' not in globals():
    parameters = pwr_description.get_default_parameters()

In [2]:

import math
from tensorflow.keras.models import load_model, model_from_json
import numpy as np
import tensorflow as tf
import os
import numpy as np

In [ ]:
from cegalprizm.pythontool import PetrelConnection

ptp = PetrelConnection()

petrel_objects = ptp.get_petrelobjects_by_guids([parameters['seismic_id']])

In [ ]:
cube=petrel_objects[0]

In [ ]:
span = 127
x = int((cube.extent.i - 128)/2)
y = int((cube.extent.j - 128)/2)
z = int((cube.extent.k - 128)/2)
arr  = cube.chunk((x,x+span),(y,y+span),(z,z+span)).as_array()

loaded_model = load_model("fseg-70.hdf5", custom_objects={'cross_entropy_balanced': tf.keras.losses.BinaryCrossentropy()})

def fault_attribute_calculator(arr):
    arr = np.rot90(arr, 1, (0,2))
    arr = (arr - arr.mean())/arr.std()
    n1, n2, n3 = 128, 128, 128
    gx = np.reshape(arr,(1,n1,n2,n3,1))
    Y = loaded_model.predict(gx,verbose=1)
    Y = Y.reshape((n1,n2,n3))
    return np.rot90(Y, -1, (0,2))

faults = fault_attribute_calculator(arr)

In [ ]:
cube_fault_prediction = cube.clone('cube_fault_prediction_demo_new', copy_values = False)

In [ ]:
a = 0.5
overlapp = 0.1
def apply_calculator(src_cube, dst_cube, calculator, chunk_size = (128,128,128)):
    m1, m2, m3 = src_cube.extent.i, src_cube.extent.j, src_cube.extent.k
    n1, n2, n3 = chunk_size
    
    x_count = math.ceil(2*m1/n1)
    y_count = math.ceil(2*m2/n2)
    
    z_count = math.ceil(2*m3/n3)
    for x in range(x_count):
        if ( x*(n1/(1+overlapp)) > (m1 - n1/2 -1)):
              continue
        for y in range(y_count):
            if (y*(n2/(1+overlapp)) > (m2 - n2/2 -1)):
                continue
            for z in range(z_count):
                if (z*(n3/(1+overlapp)) > (m3 - n3/2 -1) ):
                      continue
                x_offset = math.floor(min(x*(n1/(1+overlapp)), m1 - n1 -1))
                y_offset = math.floor(min(y*(n2/(1+overlapp)), m2 - n2 -1))
                z_offset = math.floor(min(z*(n3/(1+overlapp)), m3 - n3 -1))
                x_range = (x_offset + 1, x_offset+n1)
                y_range = (y_offset + 1, y_offset+n2)
                z_range = (z_offset + 1, z_offset+n3)
                src_data = src_cube.chunk(x_range, y_range, z_range).as_array()
                with dst_cube.chunk(x_range, y_range, z_range).values() as dst:
                    result = calculator(src_data)
                    mask = (dst == 0)
                    dst[mask] = result[mask]
                    dst[2:-2,2:-2,2:-2] = a*result[2:-2,2:-2,2:-2] + (1-a)*dst[2:-2,2:-2,2:-2]


In [ ]:
apply_calculator(cube, cube_fault_prediction, fault_attribute_calculator)